In [1]:
import pandas as pd
import numpy  as np
from sklearn import datasets

X, y = datasets.make_moons(10000, noise=0.15)

# Can't believe I'm doing it this way, but join doesn't work
# on numpy strings and I'm on a plane unable to lookup the
# right way to join a column to a matrix and output as CSV.
for x_i,y_i in zip(X,y):
    output = ''
    output += str(y_i)
    for j in range(0,len(x_i)):
        output += ','
        output += str(x_i[j])
    # print output

In [2]:
NObs =X.shape[0]

traindata=X[0:NObs/2,:]
validdata=X[NObs/2+1:NObs*3/4,:]
testdata=X[NObs*3/4+1:,:]

train_labels=pd.DataFrame(y[0:NObs/2])
valid_labels=pd.DataFrame(y[NObs/2+1:NObs*3/4])

test_labels=pd.DataFrame(y[NObs*3/4+1:])

train_labels.columns = ['label']
valid_labels.columns = ['label']
test_labels.columns = ['label']

In [ ]:
# prepare data for TensorFlow

num_var = 2
num_labels = 2 # 39 categories

train_dataset = np.float32(np.array(traindata))
N_obs_train=len(train_dataset)

valid_dataset = np.float32(np.array(validdata))

test_dataset = np.float32(np.array(testdata))

train_labels= np.array(pd.get_dummies(train_labels['label'].astype('category').cat.rename_categories(np.arange(0,num_labels)) ))
valid_labels= np.array(pd.get_dummies(valid_labels['label'].astype('category').cat.rename_categories(np.arange(0,num_labels)) ))
test_labels= np.array(pd.get_dummies ( test_labels['label'].astype('category').cat.rename_categories(np.arange(0,num_labels)) ))

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape)

test_dataset.dtype

In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [ ]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

def baseline_logloss(hotmatrix): #input is a hot-encoded sparse matrix, one 1 only per row, rest is 0
    nobs = len(hotmatrix)
    temp = hotmatrix.sum(axis=0)/nobs
    return(-sum(np.multiply(temp,np.log(temp))))

In [ ]:
# Adding regularization to the 1 hidden layer network
graph1 = tf.Graph()
batch_size = 128

epoch_size  = N_obs_train / batch_size
num_steps= 5 * epoch_size

import datetime
startTime = datetime.datetime.now() 

def define_and_run_batch(beta):
    
    num_RELU = 10
    
    with graph1.as_default():
      
      keep_prob = tf.placeholder(tf.float32)
      
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, num_var)) # remplace batch_size par None
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
    
      # in Kaggle competitions, we don't know the true labels of the test set
      tf_test_dataset = tf.constant(test_dataset)
  
      # Variables.
      weights_RELU = tf.Variable(
        tf.truncated_normal([num_var, num_RELU],stddev=0.1))
        
      # print(tf.shape(weights_RELU) )   
      #biases_RELU = tf.Variable(tf.zeros([num_RELU]))
      biases_RELU = tf.Variable(tf.random_normal([num_RELU],stddev=0.1))  
        
      weights_layer1 = tf.Variable(
        tf.truncated_normal([num_RELU, num_labels],stddev=0.1))
      #biases_layer1 = tf.Variable(tf.zeros([num_labels]))
      biases_layer1 = tf.Variable(tf.random_normal([num_labels],stddev=0.1))
  
      # Training computation.
      logits_RELU = tf.matmul(tf_train_dataset, weights_RELU) + biases_RELU
      RELU_vec = tf.nn.relu(logits_RELU)
      RELU_vec_drop = tf.nn.dropout(RELU_vec, keep_prob)
    
      logits_layer = tf.matmul(RELU_vec_drop, weights_layer1) + biases_layer1                  
      # loss = tf.reduce_mean(
      #        tf.nn.softmax_cross_entropy_with_logits(logits_layer, tf_train_labels))
      cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits_layer, tf_train_labels,name="cross_entropy")
      l2reg = tf.reduce_sum(tf.square(weights_RELU))+tf.reduce_sum(tf.square(weights_layer1))
      
      loss = tf.reduce_mean(cross_entropy+beta*l2reg)
      logloss = tf.reduce_mean(cross_entropy)
    
  # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.3).minimize(loss)
  
      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits_layer)
      
      valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu((tf.matmul(tf_valid_dataset, weights_RELU) + biases_RELU)),weights_layer1)+biases_layer1)
        
      # in Kaggle competitions, we don't know the true labels of the test set     
      test_prediction =tf.nn.softmax(
        tf.matmul(tf.nn.relu((tf.matmul(tf_test_dataset, weights_RELU) + biases_RELU)),weights_layer1)+biases_layer1)
                         
    with tf.Session(graph=graph1) as session:
 
      tf.initialize_all_variables().run()
      print("Initialized")
      for step in range(num_steps):
        
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch. 
        batch_data = train_dataset[offset:(offset + batch_size), :]
        
        batch_labels = train_labels[offset:(offset + batch_size)]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,
                    keep_prob:1.0}
        
        #
        if (step==num_steps-1):
            _, l, predictions_train,predictions_test,predictions_valid = session.run(
              [optimizer, logloss,train_prediction,test_prediction,valid_prediction], feed_dict=feed_dict)
        else:
            _, l, predictions_train = session.run(
              [optimizer, logloss,train_prediction], feed_dict=feed_dict)
            
        if (step % 50 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions_train, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      
      # test_acc = accuracy(test_prediction.eval(), test_labels)
      # print("Test accuracy: %.1f%%" % test_acc)

      
        
    x = datetime.datetime.now() - startTime
    print(x)
    return(predictions_test,predictions_valid,predictions_train)
    
    
RES,RES_v,RES_tr=define_and_run_batch(0.001)
N_obs_valid = validdata.shape[0]
N_obs_test  =  testdata.shape[0]
logloss_valid=-sum(sum(np.multiply(valid_labels,np.log(RES_v))))/N_obs_valid
logloss_test =-sum(sum(np.multiply( test_labels,np.log(RES  ))))/N_obs_test
print("Validation Logloss =%s" % logloss_valid)
print("Test       Logloss =%s" % logloss_test)

In [ ]:
# 4 layer network

graph1 = tf.Graph()

batch_size = 128

epoch_size  = N_obs_train / batch_size
num_steps= 2 * epoch_size #6501

import datetime
startTime = datetime.datetime.now() 

def define_and_run_batch_4_layer():
    
    

    import math as math

    initial_learning_rate_value = 0.5
    beta1 = 0.001
    beta2 = 0.001
    beta3 = 0.001
    beta4 = 0.001
    beta5 = 0.001

    graph2 = tf.Graph()
    with graph2.as_default():

       # Input data. For the training data, we use a placeholder that will be fed
       # at run time with a training minibatch.
       tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, num_var))
       tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        
       tf_valid_dataset = tf.constant(valid_dataset)
       tf_test_dataset = tf.constant(test_dataset)    
    
       # learning rate decay
       global_step = tf.Variable(0)
       learning_rate = tf.train.exponential_decay(initial_learning_rate_value, global_step, 1, 0.9999)

       # Hidden layer 1
       h1_size = 32  
       weights_h1 = tf.Variable(tf.truncated_normal([num_var, h1_size], stddev=math.sqrt(2.0/(num_var))))
       biases_h1 = tf.Variable(tf.zeros([h1_size]))
       h1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h1) + biases_h1)

       # Hidden layer 2
       h2_size = 16
       weights_h2 = tf.Variable(tf.truncated_normal([h1_size, h2_size], stddev=math.sqrt(2.0/h1_size)))
       biases_h2 = tf.Variable(tf.zeros([h2_size]))
       h2 = tf.nn.relu(tf.matmul(h1, weights_h2) + biases_h2)

       # Hidden layer 3
       h3_size = 8
       weights_h3 = tf.Variable(tf.truncated_normal([h2_size, h3_size], stddev=math.sqrt(2.0/h2_size)))
       biases_h3 = tf.Variable(tf.zeros([h3_size]))
       h3 = tf.nn.relu(tf.matmul(h2, weights_h3) + biases_h3)

       # Hidden layer 4
       h4_size = 4
       weights_h4 = tf.Variable(tf.truncated_normal([h3_size, h4_size], stddev=math.sqrt(2.0/h3_size)))
       biases_h4 = tf.Variable(tf.zeros([h4_size]))
       h4 = tf.nn.relu(tf.matmul(h3, weights_h4) + biases_h4)

       # Output layer
       weights_o = tf.Variable(tf.truncated_normal([h4_size, num_labels], stddev=math.sqrt(2.0/h4_size)))
       biases_o = tf.Variable(tf.zeros([num_labels]))
    
       # Training computation.
       logits = tf.matmul(h4, weights_o) + biases_o
       # loss = tf.reduce_mean(
       #     tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + ((beta1 * tf.nn.l2_loss(weights_h1)) + (beta2 * tf.nn.l2_loss(weights_h2)) 
       #        + (beta3 * tf.nn.l2_loss(weights_h3)) + (beta4 * tf.nn.l2_loss(weights_h4)) + (beta5 * tf.nn.l2_loss(weights_o)))
       
       cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels,name="cross_entropy")
       logloss = tf.reduce_mean(cross_entropy)
    
       loss = logloss + ((beta1 * tf.nn.l2_loss(weights_h1)) + (beta2 * tf.nn.l2_loss(weights_h2)) + (beta3 * tf.nn.l2_loss(weights_h3)) 
                         + (beta4 * tf.nn.l2_loss(weights_h4)) + (beta5 * tf.nn.l2_loss(weights_o)))
       
    
       # Optimizer.
       optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

       # Predictions for the training, validation, and test data.
       train_h1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h1) + biases_h1)
       train_h2 = tf.nn.relu(tf.matmul(train_h1, weights_h2) + biases_h2)
       train_h3 = tf.nn.relu(tf.matmul(train_h2, weights_h3) + biases_h3)
       train_h4 = tf.nn.relu(tf.matmul(train_h3, weights_h4) + biases_h4)
       train_logits = tf.matmul(train_h4, weights_o) + biases_o
       train_prediction = tf.nn.softmax(train_logits)
  
       valid_h1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h1) + biases_h1)
       valid_h2 = tf.nn.relu(tf.matmul(valid_h1, weights_h2) + biases_h2)
       valid_h3 = tf.nn.relu(tf.matmul(valid_h2, weights_h3) + biases_h3)
       valid_h4 = tf.nn.relu(tf.matmul(valid_h3, weights_h4) + biases_h4)
       valid_logits = tf.matmul(valid_h4, weights_o) + biases_o
       valid_prediction = tf.nn.softmax(valid_logits)
    
       test_h1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_h1) + biases_h1)
       test_h2 = tf.nn.relu(tf.matmul(test_h1, weights_h2) + biases_h2)
       test_h3 = tf.nn.relu(tf.matmul(test_h2, weights_h3) + biases_h3)
       test_h4 = tf.nn.relu(tf.matmul(test_h3, weights_h4) + biases_h4)
       test_logits = tf.matmul(test_h4, weights_o) + biases_o
       test_prediction = tf.nn.softmax(test_logits)
  
        
   

# executing the graph     
        
        
    with tf.Session(graph=graph2) as session:
 
      tf.initialize_all_variables().run()
      print("Initialized")
      for step in range(num_steps):
        
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch. 
        batch_data = train_dataset[offset:(offset + batch_size), :]
        
        batch_labels = train_labels[offset:(offset + batch_size)]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        #
        if (step==num_steps-1):
            _, l, predictions_train,predictions_test,predictions_valid = session.run(
              [optimizer, logloss,train_prediction,test_prediction,valid_prediction], feed_dict=feed_dict)
        else:
            _, l, predictions_train = session.run(
              [optimizer, logloss,train_prediction], feed_dict=feed_dict)
            
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions_train, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      
      # test_acc = accuracy(test_prediction.eval(), test_labels)
      # print("Test accuracy: %.1f%%" % test_acc)
        
    x = datetime.datetime.now() - startTime
    print(x)
    return(predictions_test,predictions_valid)
    
    
    
RES,RES_v=define_and_run_batch_4_layer()

N_obs_valid = validdata.shape[0]
N_obs_test  =  testdata.shape[0]
logloss_valid=-sum(sum(np.multiply(valid_labels,np.log(RES_v))))/N_obs_valid
logloss_test =-sum(sum(np.multiply( test_labels,np.log(RES  ))))/N_obs_test
print("Validation Logloss =%s" % logloss_valid)
print("Test       Logloss =%s" % logloss_test)

In [ ]:
print(baseline_logloss(train_labels))
print(baseline_logloss(valid_labels))

In [ ]:
print(accuracy(RES_tr,train_labels))
print(accuracy(RES_v ,valid_labels))
print(accuracy(RES   , test_labels))

In [3]:
import matplotlib.pyplot as plt


In [5]:
df=pd.DataFrame(testdata)
df.columns=['X','Y']
df['pred']=np.argmax(RES,1)
df['label']=y[NObs*3/4+1:]

def confusion (pred,label):
    if    (pred==1) and (label==1): return ('true_pos')
    elif  (pred==1) and (label==0): return ('false_pos')
    elif  (pred==0) and (label==0): return ('true_neg')
    else                          : return ('false_neg')

def myfun(row):
    return(confusion(row['pred'],row['label']))
df['class']=df.apply(myfun,axis=1)
df

NameError: name 'RES' is not defined

In [ ]:
plt.plot(df['X'],df['Y'],'ro')
plt.show()

In [ ]:
grouped = df.groupby('class')

In [ ]:
# Plot
fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
for name, group in grouped:
    ax.plot(group.X, group.Y, marker='o', linestyle='', ms=12, label=name)
ax.legend()

plt.show()

In [4]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(traindata)

In [5]:
distances,indices = nbrs.kneighbors(testdata)
indices
#distances

array([[2692, 1153],
       [1737,  699],
       [ 524,  522],
       ..., 
       [4210, 1785],
       [3698, 1160],
       [1061, 2043]])

numpy.ndarray

In [9]:
from sklearn.neighbors import KNeighborsClassifier
neigh=KNeighborsClassifier(n_neighbors=3,metric='euclidean')
y=np.array(train_labels).reshape(5000,)
neigh.fit(traindata,train_labels)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [10]:
neigh.predict(testdata)
neigh.score(testdata,test_labels)

0.98719487795118044

In [26]:
def mydist(x,y):
    z=x-y
    return(z[0]**2+ 2*z[1]**2)

In [30]:
x=np.array([1,1])
y=np.array([2,2])
z=x-y

3

In [31]:
neigh=KNeighborsClassifier(n_neighbors=3,metric=mydist)

In [32]:
neigh.fit(traindata,train_labels)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  if __name__ == '__main__':


KNeighborsClassifier(algorithm='auto', leaf_size=30,
           metric=<function mydist at 0x1098a6140>, metric_params=None,
           n_jobs=1, n_neighbors=3, p=2, weights='uniform')

In [33]:
neigh.predict(testdata)
neigh.score(testdata,test_labels)

0.98799519807923164

In [6]:
from sklearn.cluster import KMeans
km=KMeans(n_clusters=2,n_init=3)
km.fit(traindata)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=2, n_init=3,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [7]:
km.predict(traindata)

array([1, 0, 1, ..., 1, 0, 1], dtype=int32)

In [8]:
df1=pd.DataFrame(traindata)
df1.columns=['X','Y']
df1['cluster']=km.predict(traindata)
grouped1 = df1.groupby('cluster')
# Plot
fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
for name, group in grouped1:
    ax.plot(group.X, group.Y, marker='o', linestyle='', ms=12, label=name)
ax.legend()

plt.show()

In [9]:
df1

,X,Y,cluster
0,-1.060217,0.209108,1
1,1.439533,-0.329381,0
2,0.559930,0.847939,1
3,0.549476,0.405953,1
4,-0.036195,0.359942,1
5,0.917799,-0.001895,0
6,0.266213,-0.110192,1
7,-0.301464,1.084138,1
8,-0.878780,0.177226,1
9,0.035439,0.111614,1


In [34]:
2^4

6